# Get Activities of Note
Get days with more intense activities over the past 100 days. The purpose of this is to identify days worth studying the heart rate data from

In [80]:
from tqdm import tqdm_notebook as tqdm
from time import sleep
import pandas as pd
import datetime as dt
import json
import fitbit
import os

Load the Fitbit client

In [17]:
fb_client = fitbit.Fitbit(**json.load(open('client-data.json')))

## Load in the Activities
Get recent logged activities using [Fitbit's API](https://dev.fitbit.com/reference/web-api/activity/#get-activity-logs-list)

In [18]:
reply = fb_client.make_request("https://api.fitbit.com/1/user/-/activities/list.json?afterDate=2017-07-14&offset=0&limit=20&sort=desc")

Load in all of the days

In [88]:
def get_all_activities(previous_days=200, minimum_length=15):
    """Get all recent activities of note
    
    :param previous_days: int, number of days before today to study
    :param minimum_length: int, minimum minutes for activity for it to count
    :return: all relevant activities"""
    
    # Determine start date
    start_date = (dt.date.today() - dt.timedelta(previous_days)).strftime('%Y-%m-%d')
    
    # Collect all activities
    activities = []
    url = "https://api.fitbit.com/1/user/-/activities/list.json?afterDate=%s&offset=0&limit=20&sort=desc"%start_date
    while len(url) > 0:
        failures = 0
        try:
            reply = fb_client.make_request(url)
        except:
            failures += 1
            if failures > 5:
                raise
            sleep(1) # Sometimes it does not work
            continue
        activities += reply['activities']
        url = reply['pagination']['next']
        
    # Get the longer ones
    good_acts = []
    for activity in activities:
        if not 'heartRateZones' in activity:
            continue # Some do not have HR monitoring
        length = pd.DataFrame(activity['heartRateZones']).loc[1:, 'minutes'].sum()
        if length > minimum_length:
            good_acts.append(activity)
        
    return good_acts

In [89]:
activities = get_all_activities()
print('Found %d activities'%len(activities))

Found 145 activities


## Save the Unique Dates
Find all days with a relevant activity

In [90]:
dates = sorted(set([a['startTime'][:10] for a in activities]))
print('Found %d unique dates'%len(dates))

Found 91 unique dates


In [91]:
json.dump(dates, open(os.path.join('data', 'activity-dates.json'), 'w'))